In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import string

In [2]:

input_text = '''When the modern Olympics began in 1896, the initiators and organizers ---- looking for a great popularizing event, recalling the ancient glory of Greece. The idea of a marathon race came from Michel Breal, who wanted the event to feature in the first modern Olympic Games in 1896 in Athens. This idea was heavily supported by Pierre de Coubertin, the founder of the modern Olympics, as well as by the Greeks. The Greeks staged a selection race for the Olympic marathon on 10 March 1896 that ---- won by Charilaos Vasilakos in 3 hours and 18 minutes (with the future winner of the introductory Olympic Games marathon coming in fifth). The winner of the first Olympic Marathon, on 10 April 1896 (a male-only race), was Spyridon "Spyros" Louis, a Greek water-carrier, in 2 hours 58 minutes and 50 seconds. The women's marathon ---- introduced at the 1984 Summer Olympics (Los Angeles, USA) and ---- won by Joan Benoit of the United States with a time of 2 hours 24 minutes and 52 seconds. Since the modern games were founded, it has become a tradition for the men's Olympic marathon to be the last event of the athletics calendar, with a finish inside the Olympic stadium, often within hours of, or even incorporated into, the closing ceremonies. The marathon of the 2004 Summer Olympics revived the traditional route from Marathon to Athens, ending at Panathinaiko Stadium, the venue for the 1896 Summer Olympics. Since the modern games ---- founded, it has become a tradition for the men's Olympic marathon to be the last event of the athletics calendar, with a finish inside the Olympic stadium, often within hours of, or even incorporated into, the closing ceremonies. The marathon of the 2004 Summer Olympics revived the traditional route from Marathon to Athens, ending at Panathinaiko Stadium, the venue for the 1896 Summer Olympics. The Olympic men's record ---- 2:06:32.'''

In [3]:
input_text

'When the modern Olympics began in 1896, the initiators and organizers ---- looking for a great popularizing event, recalling the ancient glory of Greece. The idea of a marathon race came from Michel Breal, who wanted the event to feature in the first modern Olympic Games in 1896 in Athens. This idea was heavily supported by Pierre de Coubertin, the founder of the modern Olympics, as well as by the Greeks. The Greeks staged a selection race for the Olympic marathon on 10 March 1896 that ---- won by Charilaos Vasilakos in 3 hours and 18 minutes (with the future winner of the introductory Olympic Games marathon coming in fifth). The winner of the first Olympic Marathon, on 10 April 1896 (a male-only race), was Spyridon "Spyros" Louis, a Greek water-carrier, in 2 hours 58 minutes and 50 seconds. The women\'s marathon ---- introduced at the 1984 Summer Olympics (Los Angeles, USA) and ---- won by Joan Benoit of the United States with a time of 2 hours 24 minutes and 52 seconds. Since the mo

In [4]:
#nltk.download('punkt')
sentences = nltk.sent_tokenize(input_text)

In [5]:
def get_sentences_with_blanks(input_text):
    '''Extracts sentences with blanks from the entire corpus. '''
    
    #nltk.download('punkt')
    sentences = nltk.sent_tokenize(input_text)
    docs = [sent for sent in sentences if ' ---- ' in sent ]
    new_docs = []
    for doc in docs:
        window = re.findall('[a-zA-Z,\']+ ---- [a-zA-Z,\']+',doc)
        if len(window)>1:
            for i in range(len(window)):
                new_docs.append(doc)
        else:
            new_docs.append(doc)
    
    return new_docs

In [6]:
print(get_sentences_with_blanks(input_text))       

['When the modern Olympics began in 1896, the initiators and organizers ---- looking for a great popularizing event, recalling the ancient glory of Greece.', 'The Greeks staged a selection race for the Olympic marathon on 10 March 1896 that ---- won by Charilaos Vasilakos in 3 hours and 18 minutes (with the future winner of the introductory Olympic Games marathon coming in fifth).', "The women's marathon ---- introduced at the 1984 Summer Olympics (Los Angeles, USA) and ---- won by Joan Benoit of the United States with a time of 2 hours 24 minutes and 52 seconds.", "The women's marathon ---- introduced at the 1984 Summer Olympics (Los Angeles, USA) and ---- won by Joan Benoit of the United States with a time of 2 hours 24 minutes and 52 seconds.", "Since the modern games ---- founded, it has become a tradition for the men's Olympic marathon to be the last event of the athletics calendar, with a finish inside the Olympic stadium, often within hours of, or even incorporated into, the clo

In [7]:
#nltk.download('averaged_perceptron_tagger')
def tag_docs(new_docs):
    '''Tags the docs/sentences which have blanks with various parts-of-speech tags in nltk.'''
    
    tokenized_docs = []
    tagged_docs = []
    for doc in new_docs:
        tokens = nltk.word_tokenize(doc)
        tokenized_docs.append(tokens) 
        tagged_docs.append(nltk.pos_tag(tokens))
    
    return tagged_docs

In [8]:
new_docs = get_sentences_with_blanks(input_text)
tagged_docs = tag_docs(new_docs)
print(tagged_docs)

[[('When', 'WRB'), ('the', 'DT'), ('modern', 'JJ'), ('Olympics', 'NNPS'), ('began', 'VBD'), ('in', 'IN'), ('1896', 'CD'), (',', ','), ('the', 'DT'), ('initiators', 'NNS'), ('and', 'CC'), ('organizers', 'NNS'), ('--', ':'), ('--', ':'), ('looking', 'VBG'), ('for', 'IN'), ('a', 'DT'), ('great', 'JJ'), ('popularizing', 'NN'), ('event', 'NN'), (',', ','), ('recalling', 'VBG'), ('the', 'DT'), ('ancient', 'JJ'), ('glory', 'NN'), ('of', 'IN'), ('Greece', 'NNP'), ('.', '.')], [('The', 'DT'), ('Greeks', 'NNP'), ('staged', 'VBD'), ('a', 'DT'), ('selection', 'NN'), ('race', 'NN'), ('for', 'IN'), ('the', 'DT'), ('Olympic', 'NNP'), ('marathon', 'NN'), ('on', 'IN'), ('10', 'CD'), ('March', 'NNP'), ('1896', 'CD'), ('that', 'WDT'), ('--', ':'), ('--', ':'), ('won', 'VBN'), ('by', 'IN'), ('Charilaos', 'NNP'), ('Vasilakos', 'NNP'), ('in', 'IN'), ('3', 'CD'), ('hours', 'NNS'), ('and', 'CC'), ('18', 'CD'), ('minutes', 'NNS'), ('(', '('), ('with', 'IN'), ('the', 'DT'), ('future', 'JJ'), ('winner', 'NN'), (

In [43]:
def find_verbs_nouns(tagged_docs):
    '''Specifically extracts nouns and verb tags from the previously tagged documents.
       Verb tags are going to be used to extract tense information from the sentence whereas noun tags will be
       used to find whether the subjects in the sentence are plural or singular.'''
    
    verb_tags_list = []
    noun_tags_list = []
    for doc in tagged_docs:
        tags = ' '.join([tag[1] for tag in doc])
        verb_tags = re.findall('(VB\w?)',str(tags))
        verb_tags_list.append(verb_tags)
        noun_tags = re.findall('(NN\w*)',str(tags))
        noun_tags_list.append(noun_tags)
        #print(verb_tags)
        #print(noun_tags)
    
    return verb_tags_list,noun_tags_list

In [44]:
verb_tags_list,noun_tags_list = find_verbs_nouns(tag_docs(new_docs))
print('Verb Tags: ',verb_tags_list)
print('Noun Tags: ',noun_tags_list)

Verb Tags:  [['VBD', 'VBG', 'VBG'], ['VBD', 'VBN', 'VBP', 'VBG'], ['VBD', 'VBN'], ['VBD', 'VBN'], ['VBD', 'VBZ', 'VBN', 'VB', 'VBN'], []]
Noun Tags:  [['NNPS', 'NNS', 'NNS', 'NN', 'NN', 'NN', 'NNP'], ['NNP', 'NN', 'NN', 'NNP', 'NN', 'NNP', 'NNP', 'NNP', 'NNS', 'NNS', 'NN', 'NN', 'NNP', 'NNP', 'NN'], ['NNS', 'NN', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNPS', 'NN', 'NNS', 'NNS', 'NNS'], ['NNS', 'NN', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNPS', 'NN', 'NNS', 'NNS', 'NNS'], ['NNS', 'NN', 'NNS', 'NNP', 'NN', 'NN', 'NNS', 'NN', 'NNP', 'NN', 'NNS', 'NN', 'NNS'], ['NNP', 'NNS', 'NN']]


In [45]:
def determine_tense(verb_tags_list):
    '''Classifies a sentence based on tense.The tense is determined by counting the number of occurences of each
    type of tense in the sentence using the POS tags.
    Returns a dictionary of tuples with counts of (past,present) verbs respectively'''
    
    tense_data = {}
    i = 1 
    for tags in verb_tags_list:
        present = 0
        past = 0

        for tag in tags:
            if tag == 'VBD' or tag == 'VBN':
                past += 1
            elif tag == 'VBG' and past>0:
                past += 1
            elif tag == 'VB' or tag == 'VBP' or tag == 'VBZ':
                present += 1
            elif tag == 'VBG' and present>0:
                present += 1

        tense_data[i] = (past,present)
        i += 1
    
    return tense_data

In [46]:
tense_data = determine_tense(verb_tags_list)
print(tense_data)

{1: (3, 0), 2: (3, 1), 3: (2, 0), 4: (2, 0), 5: (3, 2), 6: (0, 0)}


In [47]:
def determine_singular(noun_tags_list):
    '''Returns a dictionary of tuples with counts of (singular,plural) nouns respectively'''
    noun_data = {}
    j = 1
    for tags in noun_tags_list:
        singular = 0
        plural = 0

        for tag in tags:
            if tag in ['NN']:
                singular += 1
            elif tag in ['NNS']:
                plural += 1

        noun_data[j] = (singular,plural)
        j += 1
        
    return noun_data

In [48]:
noun_data = determine_singular(noun_tags_list)
print(noun_data)

{1: (3, 2), 2: (6, 2), 3: (2, 4), 4: (2, 4), 5: (6, 5), 6: (1, 1)}


In [63]:
def get_context_info(tense_data,noun_data):
    '''Combines the information of tense and noun of a sentence into one nested dictionary as context info.
    The nested dictionary consists n dictionary with each having two keys (past,singular) with boolean values.'''
    
    context_data = {}
    for k,v in tense_data.items():
        cont_dict = {}
        if v[0]>v[1]:
            cont_dict['past'] = True
        else:
            cont_dict['past'] = False

        context_data[k] = cont_dict    

    for k,v in noun_data.items():

        if v[0]>=v[1]:
            context_data[k]['singular'] = True
        else:
            context_data[k]['singular'] = False

    return context_data



In [64]:
context_data = get_context_info(tense_data,noun_data)
print(context_data)

{1: {'past': True, 'singular': True}, 2: {'past': True, 'singular': True}, 3: {'past': True, 'singular': False}, 4: {'past': True, 'singular': False}, 5: {'past': True, 'singular': True}, 6: {'past': False, 'singular': True}}


In [81]:
def before_and_after_blank_info():
    '''Gets the pos tags for words before and after the blank.This information will be used as the primary base for
    determining the answer. If some information is missing related to tense or pluarality, context_dict will be referred.'''
    docs = [sent for sent in sentences if ' ---- ' in sent ]
    k=1
    before_blank = []
    after_blank = []
    data = {}
    for doc in docs:
        windows = re.findall('[a-zA-Z0-9,\']+ ---- [a-zA-Z0-9,\']+',doc)
        #print(windows)
        for window in windows:
            window_tokens = re.split(' ---- ',window)
            tagged = nltk.pos_tag(window_tokens)
            before_blank.append(tagged[0])
            after_blank.append(tagged[1])
            
            #print(tagged)
            tag_dict = {}
            for tags in tagged:
                if tags[1] in ['NN','NNP']:
                    tag_dict['singular'] = True
                elif tags[1] in ['NNS','NNPS']:
                    tag_dict['singular'] = False
                elif tags[1] in ['VBD','VBN']:
                    tag_dict['past'] = True
                elif tags[1] in ['VB','VBZ']:
                    tag_dict['past'] = False


            data[k] = tag_dict
            k+=1
    
    return data,after_blank,before_blank

In [82]:
data,after_blank,before_blank = before_and_after_blank_info()
print('Data: ',data)
print('Before blank pos: ',before_blank)
print('After blank pos: ',after_blank)

Data:  {1: {'singular': False}, 2: {'past': True}, 3: {'singular': True, 'past': True}, 4: {'past': True}, 5: {'singular': False}, 6: {'singular': True}}
Before blank pos:  [('organizers', 'NNS'), ('that', 'DT'), ('marathon', 'NN'), ('and', 'CC'), ('games', 'NNS'), ('record', 'NN')]
After blank pos:  [('looking', 'VBG'), ('won', 'VBD'), ('introduced', 'VBD'), ('won', 'VBD'), ('founded,', 'VBP'), ('2', 'CD')]


In [83]:
# combine data and context_data
for k,v in data.items():
    if 'singular' not in data[k].keys():
        data[k]['singular'] = context_data[k]['singular']
    elif 'past' not in data[k].keys():
        data[k]['past'] = context_data[k]['past']
    
data

{1: {'past': True, 'singular': False},
 2: {'past': True, 'singular': True},
 3: {'past': True, 'singular': True},
 4: {'past': True, 'singular': False},
 5: {'past': True, 'singular': False},
 6: {'past': False, 'singular': True}}

In [84]:
answers = {}
#print(data)
for i in range(0,len(before_blank)):
    if before_blank[i][0] == 'I':
        answers[i+1] = 'am'
    elif before_blank[i][1] == 'MD' or before_blank[i][0] in ['can\'t','won\'t','wasn\'t','isn\'t']:
        answers[i+1] = 'be'
    elif before_blank[i][0] in ['has','have','had','am']:
        answers[i+1] = 'been'
    elif before_blank[i][0] in ['is','was','were']:
        answers[i+1] = 'being'
    elif data[i+1]['past'] == True and data[i+1]['singular'] == True:
        answers[i+1] = 'was'
    elif data[i+1]['past'] == True and data[i+1]['singular'] == False:
        answers[i+1] = 'were'
    elif data[i+1]['past'] == False and data[i+1]['singular'] == True:
        answers[i+1] = 'is'
    elif data[i+1]['past'] == False and data[i+1]['singular'] == False:
        answers[i+1] = 'are'
    
answers
  
        
          

{1: 'were', 2: 'was', 3: 'was', 4: 'were', 5: 'were', 6: 'is'}

In [85]:
for k,v in answers.items():
    print(v)

were
was
was
were
were
is
